In [1]:
########################################################
# GEO419 Projekt Nr. 2
# Download of Imagery from the Thuringian Geoportal
# Marcel Felix & Friederike Metz
# from Sept. 2021 to Feb. 2022
########################################################

# Importblock

In [1]:
import os
from osgeo import ogr,osr,gdal
import geopandas
import shapely
import zipfile
from shapely.geometry import Polygon, LineString, Point
import requests
import ipyleaflet as ip
from ipywidgets import Layout
import pandas as pd
import utm


# User's choices

In [2]:
# 1. Please select your shapefile and ID-file here
shapefile = "../Data/shape/shape.shp"

# specify the file with Server-IDs for the grid tiles
idfile = "../Data/idlist.txt"

In [3]:
# 2. Please choose your dem-years (2010-2013 / 2014-2019 / 2020-2025)
dem_year = "2014-2019"

if dem_year == "2010-2013":
    request = "https://geoportal.geoportal-th.de/hoehendaten/Uebersichten/Stand_2010-2013.zip"
if dem_year == "2014-2019":
    request = "https://geoportal.geoportal-th.de/hoehendaten/Uebersichten/Stand_2014-2019.zip"
if dem_year == "2020-2025":
    request = "https://geoportal.geoportal-th.de/hoehendaten/Uebersichten/Stand_2020-2025.zip"
    # ^These are the download-links for the dem tile polygons
# and your Orthophoto year
year = "2019"
    # unique years are: 1997, 2008, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020

In [4]:
# Automated DOWNLOAD OF TILE POLYGONS
if not os.path.exists("../Data/"):
    os.mkdir("../Data/")
if os.path.exists("../Data/Stand_"+dem_year) == False:
    os.mkdir("../Data/Stand_"+dem_year)

gridfile = "../Data/Stand_"+dem_year+"/Stand_"+dem_year+".zip"

if os.path.exists(gridfile) == False:
    grid_polygons = requests.get(request)
    with open(gridfile, "wb") as file:
            file.write(grid_polygons.content)

with zipfile.ZipFile(gridfile, 'r') as zip_ref:
    zip_ref.extractall("../Data/Stand_"+dem_year)
os.remove(gridfile)
            

In [5]:
# 3. Please choose your desired file format (dgm / dom / las)
dem_format = "dgm"

# Get Tiles 

In [6]:
# 4. Intersection of shapefile with dem tile polygons
polygon = geopandas.read_file(shapefile)
if dem_year == "2010-2013":
    grid_path = "../Data/Stand_2010-2013/DGM2_2010-2013_Erfass-lt-Meta_UTM32-UTM_2014-12-10.shp"
elif dem_year == "2014-2019":
    grid_path = "../Data/Stand_2014-2019/DGM1_2014-2019_Erfass-lt-Meta_UTM_2020-04-20--17127.shp"
else:
    grid_path = "../Data/Stand_2020-2025/DGM1_2020-2025_Erfass-lt-Meta_UTM_2021-03--17127.shp"
    
grid = geopandas.read_file(grid_path)

# intersect and fill list
tiles = []
for poly in grid.iterrows():
    geom = poly[1].geometry
    for poly2 in polygon.iterrows():
        geom2 = poly2[1].geometry
        
        if geom.overlaps(geom2) == True:
            tiles.append(poly[1].NAME)


In [7]:
# 5. dem tile names
tiles 

['680_5643', '680_5644', '681_5643', '681_5644']

# Download request 

In [9]:
# 6. Download of dem tiles
for tile in tiles:
    # creating request url
    if dem_format == "dgm" or dem_format == "dom":
        request = "https://geoportal.geoportal-th.de/hoehendaten/"+dem_format.upper()+"/"+dem_format+"_"+dem_year+"/"+dem_format+"1_"+tile+"_1_th_"+dem_year+".zip"
    elif dem_format == "las": 
        request = "https://geoportal.geoportal-th.de/hoehendaten/"+dem_format.upper()+"/"+dem_format+"_"+dem_year+"/"+dem_format+"_"+tile+"_1_th_"+dem_year+".zip+"
    
    dem_load = requests.get(request)
    # download the file to data folder
    if not os.path.exists("../Data/dem/"):
        os.mkdir("../Data/dem/")
    current_dem = "../Data/dem/"+tile+".zip"
    demzip = []
    demzip.append(current_dem)
    with open(current_dem, "wb") as file:
        file.write(dem_load.content)
        
    # unzipping
    with zipfile.ZipFile(current_dem, 'r') as zip_ref:
        zip_ref.extractall("../Data/dem/")


In [19]:
# 7. Download of op tiles
# create data from lookup file
lookup = pd.read_csv(idfile, header=0, delimiter=',')

idlist = []

# filter lookup table by year
lookup = lookup[lookup["year"] == int(year)]

# tiles from 2018 and earlier are grouped in four; only even numbers for x and y are registered with an ID
# therefore we need to make sure, we catch all the tiles inbetween 
# only the lower left tile out of the four (where x&y are even) has an ID
if int(year) <= 2018:
    tiles_check = tiles.copy()      # a copy to iterate over while the original is being altered
    for tile in tiles_check:
        tile_x, tile_y = tile.split("_")
        tile_x, tile_y = int(tile_x), int(tile_y)
        
        if ((tile_x & 1) + (tile_y & 1)) == 1:              # if either x or y is uneven, then correct their coordinates to find the matching ID
            if tile_x & 1 == 1:
                tile_x, tile_y = str(tile_x-1), str(tile_y)
            elif tile_y & 1 == 1:
                tile_y, tile_x = str(tile_y-1), str(tile_x)
                
        elif (tile_x & 1) + (tile_y & 1) == 2:              # if both are uneven
            tile_x, tile_y = str(tile_x-1), str(tile_y-1)
        
        else: continue
        
        # first make sure, the "wrong" tile is dropped from list    
        tiles.remove(tile)  
        # at last, append the newly found tile back into the list
        new_tile = tile_x + "_" + tile_y
        if not new_tile in tiles:  # but only if it's not already there!
            tiles.append(new_tile)

# now find the matching IDs
for tile in tiles:
    tile_match = lookup[lookup["tilename"] == tile]
    id_match = tile_match.values[0][0] # first row, first column
    idlist.append(id_match)

# download the identified files
for tile_id in idlist:
    # create dir if necessary
    if os.path.exists("../Data/op/") == False:
        os.mkdir("../Data/op/")
    
    # download the file
    current_op = "../Data/op/" + str(tile_id) + ".zip"
    # if already there, unzip and then continue with the next tile
    if os.path.exists(current_op):
        with zipfile.ZipFile(current_op, 'r') as zip_ref:
            zip_ref.extractall("../Data/op/")
        continue
    
    op_load = requests.get("https://geoportal.geoportal-th.de/gaialight-th/_apps/dladownload/download.php?type=op&id=" + str(tile_id))
    op_zip = []
    op_zip.append(current_op)
    
    # safe to file
    with open(current_op, "wb") as file:
        file.write(op_load.content)

    with zipfile.ZipFile(current_op, 'r') as zip_ref:
        zip_ref.extractall("../Data/op/")


# Converting XYZ to tif and Merging DEMs and OP 


In [20]:
# 8. Conversion & Merge of DEM

# find the DEM xyz files
demlist = []
for file in os.scandir("../Data/dem/"):
    if file.name.endswith(".xyz"):
        demlist.append(file.path)

# convert to tif
for xyz in demlist:
    outname = xyz.replace(".xyz","")
    gdal.Translate(outname + ".tif", xyz, outputSRS="EPSG:25832")

# merging of DEMs
# build virtual raster and convert to geotiff
demlist = [file.replace(".xyz",".tif") for file in demlist]
vrt = gdal.BuildVRT("../Data/dem/merged.vrt", demlist)
gdal.Translate("../Data/dem/mergedDEM.tif", vrt, xRes = 1, yRes = -1)
   

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000019510084CC0> >

In [21]:
# 9.Merge of OP

# find the OP files
oplist = []
for file in os.scandir("../Data/op/"):
    if file.name.endswith(".tif"):
        oplist.append(file.path)

# merging of OPs
vrt = gdal.BuildVRT("../Data/dem/merged.vrt", oplist)
gdal.Translate("../Data/op/mergedOP.tif", vrt, xRes = 0.2, yRes = -0.2)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000019510092120> >


# Clipping all to custom extent 


In [22]:
# 10. clipping with shapefile

inraster = "../Data/op/mergedOP.tif"
clipraster = "../Data/op/mergedOP_clip.tif"
options = {"cutlineDSName":shapefile, "cropToCutline":True, "dstNodata":0, "dstSRS":"EPSG:25832"}
result = gdal.Warp(clipraster, inraster, **options)
result = None

inraster = "../Data/dem/mergedDEM.tif"
clipraster = "../Data/dem/mergedDEM_clip.tif"
options = {"cutlineDSName":shapefile, "cropToCutline":True, "dstNodata":0, "dstSRS":"EPSG:25832"}
result = gdal.Warp(clipraster, inraster, **options)
result = None

# Visualisation 


In [33]:
# 11. initialize map and load open street map

# convert AOI centroid utm to latlon
centering = utm.to_latlon(polygon.centroid.x[0], polygon.centroid.y[0], 32, 'U')

# add a OSM basemap
map = ip.Map(basemap = ip.basemaps.CartoDB.Positron, center = (centering[0], centering[1]), 
             zoom = 12, layout = Layout(width = '100%', height = '700px'))

wms = ip.WMSLayer(url = 'https://tile.openstreetmap.org/${z}/${x}/${y}.png', format = 'image/png', transparent = True)
map.add_layer(wms)

# add a marker
map_marker = ip.Marker(location=(centering[0], centering[1]))
map.add_layer(map_marker)

In [29]:
# 12. overlay Data
# adding the AOI polygon 
map_poly = polygon.to_crs(4326)
map_polygon = ip.GeoData(geo_dataframe = map_poly,
                   style = {'color': 'black', 'fillColor': '#3366cc', 'opacity':0.4, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.05},
                   hover_style = {'fillColor': 'red' , 'fillOpacity': 0.05},
                   name = 'AOE')
map.add_layer(map_polygon)




In [25]:
# create OP and DEM layers to show

gdal.Translate("../Data/op/mergedOP_clip.png", "../Data/op/mergedOP_clip.tif", outputSRS="EPSG:4326")
gdal.Translate("../Data/dem/mergedDEM_clip.png", "../Data/dem/mergedDEM_clip.tif", outputSRS="EPSG:4326")


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000019510084AE0> >

In [36]:
# adding an OP layer
# !!! Adjust the access port (e.g. 8888) to what it says in your browser
op_raster = ip.ImageOverlay(
    url = "http://localhost:8889/tree/Abschlussaufgabe/Data/op/mergedOP_clip.png", #adjust url if necessary
    bounds = ((map_poly.bounds.miny[0], map_poly.bounds.minx[0]) , (map_poly.bounds.maxy[0],map_poly.bounds.maxx[0])) #low left & upper right
)
map.add_layer(op_raster)

In [32]:
# adding an DEM layer
# !!! Adjust the access port (e.g. 8888) to what it says in your browser
dem_raster = ip.ImageOverlay(
    url = "http://localhost:8889/tree/Abschlussaufgabe/Data/dem/mergedDEM_clip.png", #adjust url if necessary
    bounds = ((map_poly.bounds.miny[0], map_poly.bounds.minx[0]) , (map_poly.bounds.maxy[0],map_poly.bounds.maxx[0])) #low left & upper right
)
map.add_layer(dem_raster)

In [35]:
#show Map
map

Map(center=[50.92066794243591, 11.575571121873141], controls=(ZoomControl(options=['position', 'zoom_in_text',…